# Лабораторная работа №1

## Выбор набора данных и метрик

Я выбрал датасет, содержащий данные о химическом составе различных вин. Этот датасет можно использовать как для классификации, так и для регрессии. Для классификации мы можем выделить несколько классов качества вина (например, 0 - низкое качество, 1 - среднее, 2 - высокое качество). Также этот датасет не содержит пропусков, что упрощает работу.

**Классификация**: задача классификации качества вина.
**Регрессия**: задача предсказания точного значения качества вина.

**Метрики для оценки**:
 - Для классификации: Accuracy, F1-Score.
 - Для регрессии: MSE, R², MAE.

**Для классификации**:
* ```Accuracy (Точность)```: Эта метрика показывает долю правильно классифицированных примеров. Применяется, если классы сбалансированы и важна общая точность классификации.
* ```F1-Score```: Этот показатель является средним гармоническим точности и полноты. Подходит, когда классы несбалансированы, и важно минимизировать как ложноположительные, так и ложносогласные ошибки.

**Для регрессии**:
* ```Mean Squared Error (MSE)```: Среднеквадратичная ошибка используется для измерения разницы между предсказанными и реальными значениями. Это хорошая метрика для оценки точности модели в задачах регрессии.
* ``R² (Коэффициент детерминации)``: Это метрика, которая оценивает, какая доля вариации целевой переменной объясняется моделью. R² близкий к 1 означает хорошую модель.
* ```Mean Absolute Error (MAE)```: Средняя абсолютная ошибка также используется для оценки отклонений между предсказанными и реальными значениями.

**Практическая значимость**: Прогнозирование качества вина является реальной задачей, используемой в виноделии, чтобы оценить, какие химические компоненты оказывают влияние на качество.

## Алгоритм KNN

Импортируем библиотеки

In [276]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score, mean_absolute_error, classification_report
from sklearn.preprocessing import StandardScaler

Загрузка данных из датасета

In [277]:
# Загрузка данных
df = pd.read_csv('winequality-red.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Разделение данных:
* Для классификации целевая переменная y_class преобразована в бинарную (0 или 1) на основе того, если качество вина больше или равно 7, считаем его высококачественным.

* Для регрессии мы оставляем точное значение качества.

In [278]:
# Разделение на признаки и целевую переменную для классификации и регрессии
X = df.drop('quality', axis=1)

# Для классификации:
y_class = (df['quality'] >= 7).astype(int)
# Для регрессии:
y_reg = df['quality']

Разделение данных на обучающую и тестовую выборки (80% обучение, 20% тестирование)

In [279]:
X_train, X_test, y_train_class, y_test_class = train_test_split(X, y_class, test_size=0.2, random_state=42)
y_train_reg, y_test_reg = train_test_split(y_reg, test_size=0.2, random_state=42)

### ```Классификация``` с использованием KNN

In [280]:
knn_class = KNeighborsClassifier(n_neighbors=5)
knn_class.fit(X_train, y_train_class)
y_pred_class = knn_class.predict(X_test)

Оценка модели классификации

In [281]:
accuracy_classic_class = accuracy_score(y_test_class, y_pred_class)
f1_classic_class = f1_score(y_test_class, y_pred_class)
print("Бейзлайн:")
print(f"Accuracy: {accuracy_classic_class}")
print(f"F1 Score: {f1_classic_class}")

Бейзлайн:
Accuracy: 0.85625
F1 Score: 0.30303030303030304


Подбор гиперпараметров с помощью GridSearchCV

In [282]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn_class = KNeighborsClassifier()
grid_search_class = GridSearchCV(knn_class, param_grid, cv=5)
grid_search_class.fit(X_train_scaled, y_train_class)

# Лучшие параметры
print("Best parameters for classification:", grid_search_class.best_params_)

Best parameters for classification: {'metric': 'manhattan', 'n_neighbors': 11}


Обучение модели с улучшениями

In [283]:
best_knn_class = grid_search_class.best_estimator_

y_pred_class = best_knn_class.predict(X_test_scaled)
accuracy_impr_class = accuracy_score(y_test_class, y_pred_class)
f1_impr_class = f1_score(y_test_class, y_pred_class)

Вывод метрик

In [284]:
print("Улучшенный бейзлайн:")
print(f"Accuracy: {accuracy_impr_class}")
print(f"F1 Score: {f1_impr_class}")

print("\nClassification Report:")
print(classification_report(y_test_class, y_pred_class))

Улучшенный бейзлайн:
Accuracy: 0.88125
F1 Score: 0.5128205128205128

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.96      0.93       273
           1       0.65      0.43      0.51        47

    accuracy                           0.88       320
   macro avg       0.78      0.69      0.72       320
weighted avg       0.87      0.88      0.87       320



Реализуем собственную версию KNN

In [285]:
class CustomKNN:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def _compute_distance(self, x1, x2):
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError("Unsupported metric!")

    def _get_neighbors(self, x):
        distances = [self._compute_distance(x, x_train) for x_train in self.X_train]
        neighbors = np.argsort(distances)[:self.n_neighbors]
        return neighbors

    def predict_classification(self, X):
        predictions = []
        for x in X:
            neighbors = self._get_neighbors(x)
            neighbor_labels = self.y_train[neighbors]
            most_common = Counter(neighbor_labels).most_common(1)[0][0]
            predictions.append(most_common)
        return np.array(predictions)

# Обучение
knn_custom_class = CustomKNN(n_neighbors=5)
knn_custom_class.fit(X_train_scaled, y_train_class)
y_pred_custom_class = knn_custom_class.predict_classification(X_test_scaled)

# Метрики
accuracy_custom_class = accuracy_score(y_test_class, y_pred_custom_class)
f1_custom_class = f1_score(y_test_class, y_pred_custom_class)

# Вывод результатов
print("Реализация Custom KNN:")
print(f"Accuracy: {accuracy_custom_class}")
print(f"F1 Score: {f1_custom_class}")

Реализация Custom KNN:
Accuracy: 0.88125
F1 Score: 0.5128205128205128


Сравним полученные резльтаты:

In [286]:
print("Сравнение результатов:")
print(f"Бейзлайн Accuracy: {accuracy_classic_class:.4f}, Улучшенный Accuracy: {accuracy_impr_class:.4f}, Accuracy Custom KNN : {accuracy_custom_class:.4f}")
print(f"Бейзлайн F1-Score: {f1_classic_class:.4f}, Улучшенный F1-Score: {f1_impr_class:.4f}, F1-Score Custom KNN: {f1_custom_class:.4f}")


Сравнение результатов:
Бейзлайн Accuracy: 0.8562, Улучшенный Accuracy: 0.8812, Accuracy Custom KNN : 0.8812
Бейзлайн F1-Score: 0.3030, Улучшенный F1-Score: 0.5128, F1-Score Custom KNN: 0.5128


### ```Регрессия``` с использованием KNN

In [287]:
knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_train, y_train_reg)
y_pred_reg = knn_reg.predict(X_test)

Оценка модели регрессии

In [288]:
mse_сlassic = mean_squared_error(y_test_reg, y_pred_reg)
mae_classic = mean_absolute_error(y_test_reg, y_pred_reg)
r2_classic = r2_score(y_test_reg, y_pred_reg)

print("Regression Metrics:")
print(f"Mean Squared Error (MSE): {mse_сlassic}")
print(f"Mean Absolute Error (MAE): {mae_classic}")
print(f"R² Score: {r2_classic}")

Regression Metrics:
Mean Squared Error (MSE): 0.5319999999999999
Mean Absolute Error (MAE): 0.5787500000000001
R² Score: 0.18592925775938085


Подбор гиперпараметров с помощью GridSearchCV

In [289]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}
grid_search_reg = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5)
grid_search_reg.fit(X_train_scaled, y_train_reg)

# Лучшие параметры
print("Best parameters for classification:", grid_search_reg.best_params_)

Best parameters for classification: {'metric': 'manhattan', 'n_neighbors': 11}


Обучение с лучшими параметрами

In [290]:
best_knn_reg = grid_search_reg.best_estimator_

y_pred_reg = best_knn_reg.predict(X_test_scaled)
mse_improved = mean_squared_error(y_test_reg, y_pred_reg)
mae_improved = mean_absolute_error(y_test_reg, y_pred_reg)
r2_improved = r2_score(y_test_reg, y_pred_reg)

Вывод метрик для улучшеного бейзлайна

In [291]:
print("Improved Regression Metrics:")
print(f"Mean Squared Error (MSE): {mse_improved}")
print(f"Mean Absolute Error (MAE): {mae_improved}")
print(f"R² Score: {r2_improved}")

Improved Regression Metrics:
Mean Squared Error (MSE): 0.36893078512396693
Mean Absolute Error (MAE): 0.4900568181818182
R² Score: 0.43545910135097343


Реализации своей версии KNN для регрессии

In [292]:
# Реализация KNN
class CustomKNN:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def _compute_distance(self, x1, x2):
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError("Unsupported metric!")

    def _get_neighbors(self, x):
        distances = [self._compute_distance(x, x_train) for x_train in self.X_train]
        neighbors = np.argsort(distances)[:self.n_neighbors]
        return neighbors

    def predict_regression(self, X):
        predictions = []
        for x in X:
            neighbors = self._get_neighbors(x)
            neighbor_values = self.y_train[neighbors]
            predictions.append(np.mean(neighbor_values))
        return np.array(predictions)

# Обучение
knn_custom_reg = CustomKNN(n_neighbors=11, metric='manhattan')
knn_custom_reg.fit(X_train_scaled, y_train_reg)
y_pred_custom_reg = knn_custom_reg.predict_regression(X_test_scaled)

Вывод метрик для регрессии

In [293]:
# Регрессия
mse_custom = mean_squared_error(y_test_reg, y_pred_custom_reg)
mae_custom = mean_absolute_error(y_test_reg, y_pred_custom_reg)
r2_custom = r2_score(y_test_reg, y_pred_custom_reg)

print("\nCustom Regression Metrics:")
print(f"Mean Squared Error (MSE): {mse_custom}")
print(f"Mean Absolute Error (MAE): {mae_custom}")
print(f"R² Score: {r2_custom}")


Custom Regression Metrics:
Mean Squared Error (MSE): 0.36893078512396693
Mean Absolute Error (MAE): 0.4900568181818182
R² Score: 0.43545910135097343


In [294]:
print("Сравнение результатов:")

print(f"Бейзлайн MSE: {mse_сlassic:.4f}, Улучшенный MSE: {mse_improved:.4f}, Custom KNN MSE: {mse_custom:.4f}")
print(f"Бейзлайн MAE: {mae_classic:.4f}, Улучшенный MAE: {mae_improved:.4f}, Custom KNN MAE: {mae_custom:.4f}")
print(f"Бейзлайн R²: {r2_classic:.4f}, Улучшенный R²: {r2_improved:.4f}, Custom KNN R²: {r2_custom:.4f}")


Сравнение результатов:
Бейзлайн MSE: 0.5320, Улучшенный MSE: 0.3689, Custom KNN MSE: 0.3689
Бейзлайн MAE: 0.5788, Улучшенный MAE: 0.4901, Custom KNN MAE: 0.4901
Бейзлайн R²: 0.1859, Улучшенный R²: 0.4355, Custom KNN R²: 0.4355


# Лабораторная работа №2